In [ ]:
from astropy.io import fits

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

# Going through the tutorial here: https://prospect.readthedocs.io/en/latest/quickstart.html

In [ ]:
import fsps
import dynesty
import sedpy
import h5py, astropy
import numpy as np

In [ ]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
bands = "ugriz"
mcol = [f"cModelMag_{b}" for b in bands]
ecol = [f"cModelMagErr_{b}" for b in bands]
cat = SDSS.query_crossid(SkyCoord(ra=204.46376, dec=35.79883, unit="deg"), 
                         data_release=16, photoobj_fields=mcol + ecol + ["specObjID"])
shdus = SDSS.get_spectra(plate=2101, mjd=53858, fiberID=220)[0]
assert int(shdus[2].data["SpecObjID"][0]) == cat[0]["specObjID"]

In [ ]:
from sedpy.observate import load_filters
from prospect.utils.obsutils import fix_obs

filters = load_filters([f"sdss_{b}0" for b in bands])
maggies = np.array([10**(-0.4 * cat[0][f"cModelMag_{b}"]) for b in bands])
magerr = np.array([cat[0][f"cModelMagErr_{b}"] for b in bands])
magerr = np.clip(magerr, 0.05, np.inf)

obs = dict(wavelength=None, spectrum=None, unc=None, redshift=shdus[2].data[0]["z"],
           maggies=maggies, maggies_unc=magerr * maggies / 1.086, filters=filters)
obs = fix_obs(obs)

In [ ]:
from prospect.models.templates import TemplateLibrary
from prospect.models import SpecModel
model_params = TemplateLibrary["parametric_sfh"]
model_params.update(TemplateLibrary["nebular"])
model_params["zred"]["init"] = obs["redshift"]

model = SpecModel(model_params)
assert len(model.free_params) == 5
print(model)

In [ ]:
noise_model = (None, None)

In [ ]:
from prospect.sources import CSPSpecBasis
sps = CSPSpecBasis(zcontinuous=1)
print(sps.ssp.libraries)

In [ ]:
current_parameters = ",".join([f"{p}={v}" for p, v in zip(model.free_params, model.theta)])
print(current_parameters)
spec, phot, mfrac = model.predict(model.theta, obs=obs, sps=sps)
print(phot / obs["maggies"])

In [ ]:
from prospect.fitting import lnprobfn, fit_model
fitting_kwargs = dict(nlive_init=400, nested_method="rwalk", nested_posterior_thresh=0.05)
output = fit_model(obs, model, sps, optimize=False, dynesty=True, lnprobfn=lnprobfn, noise=noise_model, **fitting_kwargs)
result, duration = output["sampling"]

In [ ]:
from prospect.io import write_results as writer
hfile = "quickstart_dynesty_mcmc.h5"
writer.write_hdf5(hfile, {}, model, obs,
                 output["sampling"][0], None,
                 sps=sps,
                 tsample=output["sampling"][1],
                 toptimize=0.0)


# Using Taskfarmer

In [6]:
import pandas as pd
import csv

In [7]:
galaxies = pd.read_csv("../data/first_query.txt", nrows=1000)

In [8]:
shft_cmd = "shifter --image=audreykoz/gradient:latest" \
" --volume='/global/cscratch1/sd/eramey16/data/testrun:/gradient_boosted/exports'" \
" /opt/conda/bin/python /gradient_boosted/classify.py -r {} -d {}"

In [9]:
galaxies['run'] = [shft_cmd.format(a,b) for a,b in zip(galaxies.ra, galaxies.dec)]

In [10]:
galaxies

,ls_id,ra,dec,run
0,8796115184194014,149.285304,1.345116,shifter --image=audreykoz/gradient:latest --vo...
1,8796115184193402,149.273901,1.313647,shifter --image=audreykoz/gradient:latest --vo...
2,8796115184193181,149.291823,1.303267,shifter --image=audreykoz/gradient:latest --vo...
3,8796115278499067,149.127949,1.530269,shifter --image=audreykoz/gradient:latest --vo...
4,8796115278498990,149.121014,1.526591,shifter --image=audreykoz/gradient:latest --vo...
...,...,...,...,...
995,8796115278693161,149.765329,1.415705,shifter --image=audreykoz/gradient:latest --vo...
996,8796115278693010,149.784284,1.406019,shifter --image=audreykoz/gradient:latest --vo...
997,8796115278693406,149.765189,1.427825,shifter --image=audreykoz/gradient:latest --vo...
998,8796115278693488,149.783395,1.435049,shifter --image=audreykoz/gradient:latest --vo...


In [11]:
galaxies['run'].to_csv('../data/tasks.txt', sep='\t', header=False, index=False, quoting=3)

In [38]:
galaxies = pd.read_csv("../data/masterlens.tsv", skiprows=1, sep='\t')

In [41]:
galaxies.columns = galaxies.columns.str.replace('"', '')
galaxies.columns = galaxies.columns.str.strip()
# galaxies.apply(lambda s: s.str.replace('"', ""))
# galaxies.apply(lambda s: s.str.strip())
galaxies.columns

Index(['# system_name', 'discovery_date', 'alternate_name', 'kind_acronym',
       'discovery_acronym', 'reference_identifier', 'ra_hrs', 'ra_mins',
       'ra_secs', 'ra_coord', 'dec_degrees', 'dec_arcmin', 'dec_arcsec',
       'dec_coord', 'lensgrade', 'number_images', 'theta_e', 'theta_e_err',
       'theta_e_quality', 'z_lens', 'z_lens_err', 'z_lens_quality', 'z_source',
       'z_source_err', 'z_source_quality', 'vdisp', 'vdisp_err', 'filter_lens',
       'filter_source', 'fluxes', 'mag_lens', 'mag_source', 'morphology',
       'has_sdss', 'sdss_link', 'has_adsabs', 'adsabs_link', 'apod_link',
       'vett_status', 'released_status', 'hidden_status', 'vetted', 'released',
       'hidden', 'graphic_status', 'equinox', 'sdss_ObjID', 'sdss_specObjID',
       'time_delay0', 'time_delay1', 'lens_name'],
      dtype='object')

In [48]:
radec = galaxies[['ra_coord', 'dec_coord']]
radec.columns = ['ra', 'dec']
radec.to_csv('coords_A.csv', index=False)

In [33]:
galaxies.iloc[0]['discovery_date']

' "2008-08-01"'